# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [33]:
qtys = data.groupby(['CustomerID', 'ProductName']).sum().loc[:,['Quantity']]
qtys

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [15]:
product_qty = pd.pivot_table(data,
               index='ProductName',
               columns='CustomerID',
               values='Quantity',
               aggfunc=sum).fillna(0).transpose()
product_qty

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.0,0.0,0.0,25.0,0.0,50.0,0.0,25.0,0.0,0.0,...,0.0,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98069,0.0,0.0,0.0,25.0,0.0,25.0,0.0,0.0,0.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98159,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,50.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [17]:
squareform(pdist(product_qty, metric='euclidean'))

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [81]:
dist_eucl = 1 / (1 + squareform(pdist(product_qty, metric='euclidean')))
distances = pd.DataFrame(dist_eucl, index=product_qty.index, 
                         columns=product_qty.index)
similarities = list(distances[200].sort_values(ascending=False)[1:].nlargest(5).index)
similarities

[3317, 1072, 3535, 1920, 3909]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [82]:
qty_simil = qtys.loc[similarities]
qty_simil

Quantity
CustomerID ProductName                             
3317       Bay Leaf                               1
           Beef - Ground Medium                   1
           Beef - Rib Eye Aaa                     1
           Butter - Unsalted                      1
           Cheese - Victor Et Berthold            1
...                                             ...
3909       Wine - Charddonnay Errazuriz           2
           Wine - Fume Blanc Fetzer               1
           Wine - Magnotta, Merlot Sr Vqa         1
           Wine - Toasted Head                    1
           Wine - Valpolicella Masi               1

[262 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [66]:
top_prod = qty_simil.groupby('ProductName').sum().sort_values('Quantity', ascending=False)
top_prod

,Quantity
ProductName,
Soup - Campbells Bean Medley,4
Muffin - Carrot Individual Wrap,3
Bay Leaf,3
Pork - Kidney,3
"Pepper - Black, Whole",3
...,...
Kiwi,1
Knife Plastic - White,1
Lambcasing,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [67]:
prod_200 = product_qty.T.loc[:,[200]]
prod_200

CustomerID,200
ProductName,
Anchovy Paste - 56 G Tube,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0
Appetizer - Mushroom Tart,0.0
Appetizer - Sausage Rolls,0.0
Apricots - Dried,0.0
...,...
Yeast Dry - Fermipan,0.0
Yoghurt Tubes,0.0
"Yogurt - Blueberry, 175 Gr",1.0


In [80]:
merge = top_prod.merge(prod_200, on='ProductName')
recommendations_5 = list(merge[merge[200] == 0].head().index)

['Soup - Campbells Bean Medley',
 'Muffin - Carrot Individual Wrap',
 'Bay Leaf',
 'Pork - Kidney',
 'Wanton Wrap']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [84]:
def recommendation(customer):
    dist_eucl = 1 / (1 + squareform(pdist(product_qty, metric='euclidean')))
    distances = pd.DataFrame(dist_eucl, index=product_qty.index, columns=product_qty.index)
    similarities = list(distances[customer].sort_values(ascending=False)[1:].nlargest(5).index)
    qty_simil = qtys.loc[similarities]
    top_prod = qty_simil.groupby('ProductName').sum().sort_values('Quantity', ascending=False)
    prod_customer = product_qty.T.loc[:,[customer]]
    merge = top_prod.merge(prod_200, on='ProductName')
    recommendations_5 = list(merge[merge[200] == 0].head().index)
    return recommendations_5

In [91]:
recom_dict = {x: recommendation(x) for x in data['CustomerID'].unique()}

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [94]:
pd.DataFrame.from_dict(recom_dict, orient='index', columns=['Product1', 'Product2', 'Product3', 'Product4', 'Product5'])

,Product1,Product2,Product3,Product4,Product5
61288,Beef - Rib Eye Aaa,Jagermeister,Chicken - Soup Base,Lime Cordial - Roses,"Lamb - Pieces, Diced"
77352,Wine - Valpolicella Masi,Fuji Apples,Ocean Spray - Ruby Red,Tuna - Salad Premix,Chicken - Soup Base
40094,Bread - Italian Corn Meal Poly,Puree - Mocha,"Cheese - Brie,danish",Cookie - Dough Variety,Tuna - Salad Premix
23548,Tea - English Breakfast,Squid - Tubes / Tenticles 10/20,Spinach - Baby,Jolt Cola - Electric Blue,Sprouts - Baby Pea Tendrils
78981,Isomalt,Vanilla Beans,Lettuce - Frisee,Grouper - Fresh,Corn Meal
...,...,...,...,...,...
49005,Pickerel - Fillets,Phyllo Dough,"Pasta - Penne, Rigate, Dry",Lettuce - Spring Mix,Browning Caramel Glace
41286,Veal - Inside,Beer - Labatt Blue,"Thyme - Lemon, Fresh",Wonton Wrappers,"Veal - Inside, Choice"
85878,"Beef - Tenderlion, Center Cut",Cake - Mini Cheesecake,Wine - White Cab Sauv.on,Yoghurt Tubes,Veal - Eye Of Round
68506,Onions - Cippolini,"Veal - Brisket, Provimi,bnls",Scampi Tail,"Rosemary - Primerba, Paste",Scallops - 10/20


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [98]:
def recommendation_corr(customer):
    dist_eucl = 1 / (1 + squareform(pdist(product_qty, metric='correlation')))
    distances = pd.DataFrame(dist_eucl, index=product_qty.index, columns=product_qty.index)
    similarities = list(distances[customer].sort_values(ascending=False)[1:].nlargest(5).index)
    qty_simil = qtys.loc[similarities]
    top_prod = qty_simil.groupby('ProductName').sum().sort_values('Quantity', ascending=False)
    prod_customer = product_qty.T.loc[:,[customer]]
    merge = top_prod.merge(prod_200, on='ProductName')
    recommendations_5 = list(merge[merge[200] == 0].head().index)
    return recommendations_5

def recommendation_cb(customer):
    dist_eucl = 1 / (1 + squareform(pdist(product_qty, metric='cityblock')))
    distances = pd.DataFrame(dist_eucl, index=product_qty.index, columns=product_qty.index)
    similarities = list(distances[customer].sort_values(ascending=False)[1:].nlargest(5).index)
    qty_simil = qtys.loc[similarities]
    top_prod = qty_simil.groupby('ProductName').sum().sort_values('Quantity', ascending=False)
    prod_customer = product_qty.T.loc[:,[customer]]
    merge = top_prod.merge(prod_200, on='ProductName')
    recommendations_5 = list(merge[merge[200] == 0].head().index)
    return recommendations_5

In [96]:
recommendation_corr(61288)

['Longos - Grilled Salmon With Bbq',
 'Jagermeister',
 'Wine - Two Oceans Cabernet',
 'Wine - Fume Blanc Fetzer',
 'Tofu - Firm']

In [99]:
recommendation_cb(61288)

['Soup - Campbells Bean Medley',
 'Knife Plastic - White',
 'Sausage - Liver',
 'Butter - Unsalted',
 'Bay Leaf']